In [3]:
#pip install py3dep
#pip install pynhd
#pip install sklearn
#!pip install laspy[lazrs]
#!pip install pandas
#!pip install geopandas
#!pip install shapely
#!pip install plotly
#!pip install -U kaleido
# !pip install pdal

In [4]:
import numpy as np 
import pandas as pd
import os,sys
import laspy
# import pdal

from geopandas import gpd
from shapely.geometry import Polygon,Point
import plotly.offline as go_offline
import plotly.graph_objects as go
import plotly.express as px
sys.path.append(os.path.abspath(os.path.join('../Scripts')))
from file_handler import FileHandler
from config import Config

import warnings
warnings.filterwarnings('ignore')

In [5]:
# def get_dep_points(self, array_of_points: np.ndarray) -> gpd.GeoDataFrame: 
#     geometry_points = [Point(x, y) for x, y in zip(array_of_points[:, 0], array_of_points[:, 1])]
#     elevations = np.array(array_of_points[:, 2])

#     df = gpd.GeoDataFrame(columns=["elevation", "geometry"])
#     df['elevation'] = elevations
#     df['geometry'] = geometry_points
#     df = df.set_geometry("geometry")
#     df.set_crs(epsg=self.output_epsg, inplace=True)
#     return df

# def get_dep(self, array_data: np.ndarray) -> gpd.GeoDataFrame:
#     for i in array_data:
#         geometry_points = [Point(x, y) for x, y in zip(i["X"], i["Y"])]
#         elevations = np.array(i["Z"])

#         df = gpd.GeoDataFrame(columns=["elevation", "geometry"])
#         df['elevation'] = elevations
#         df['geometry'] = geometry_points
#         df = df.set_geometry("geometry")
#         df.set_crs(epsg=self.output_epsg, inplace=True)
#     return df

# def get_polygon_str(self, x_cord, y_cord) -> str:
#     polygon_str = 'POLYGON(('
#     for x, y in zip(list(x_cord), list(y_cord)):
#     polygon_str += f'{x} {y}, '
#     polygon_str = polygon_str[:-2]
#     polygon_str += '))'
#     return polygon_str

# def get_bound_from_polygon(self, polygon: Polygon) -> tuple:
#     polygon_df = gpd.GeoDataFrame([polygon], columns=['geometry'])
#     polygon_df.set_crs(epsg=self.output_epsg, inplace=True)
#     polygon_df['geometry'] = polygon_df['geometry'].to_crs(epsg=self.input_epsg)
#     xmin, ymin, xmax, ymax = polygon_df['geometry'][0].bounds
#     bound = Bounds(xmin, xmax, ymin, ymax)
#     x_cord, y_cord = polygon_df['geometry'][0].exterior.coords.xy
#     polygon_str = self.get_polygon_str(x_cord, y_cord)
#     return bound, polygon_str

# def get_pipeline(self, bounds: str, polygon_str: str, region: str, filename: str):
#     pipe = self._file_handler.read_json("usgs_3dep_pipeline")
#     pipe['pipeline'][0]['filename'] = Config.USGS_3DEP_PUBLIC_DATA_PATH + \
#         region + "/ept.json"
#     pipe['pipeline'][0]['bounds'] = bounds
#     pipe['pipeline'][1]['polygon'] = polygon_str
#     pipe['pipeline'][6]['out_srs'] = f'EPSG:{self.output_epsg}'
#     pipe['pipeline'][7]['filename'] = str(
#         Config.LAZ_PATH / str(filename + ".laz"))
#     pipe['pipeline'][8]['filename'] = str(
#         Config.TIF_PATH / str(filename + ".tif"))
#     return pdal.Pipeline(json.dumps(pipe))


In [6]:
try:
    inFile = laspy.read('../laz-las-json/iowa.laz')
except FileNotFoundError:
    print("File not Found")
    
lidar_points = np.array((inFile.x,inFile.y,inFile.z,inFile.intensity,
               inFile.raw_classification,inFile.scan_angle_rank)).transpose()

#Transform to pandas DataFrame
lidar_df=pd.DataFrame(lidar_points)

In [7]:
lidar_df.head()

,0,1,2,3,4,5
0,446179.84,4654067.44,277.06,96.0,2.0,10.0
1,446181.77,4654067.16,278.33,25.0,2.0,10.0
2,446183.40,4654067.06,276.64,88.0,2.0,10.0
3,446169.57,4654067.28,280.31,18.0,2.0,11.0
4,446158.14,4654067.66,279.39,51.0,2.0,11.0


In [8]:
#rename column names 
lidar_df.columns = ["x","y","z","intensity","raw_classification","scan_angle_rank"]

In [9]:
lidar_df.head()

,x,y,z,intensity,raw_classification,scan_angle_rank
0,446179.84,4654067.44,277.06,96.0,2.0,10.0
1,446181.77,4654067.16,278.33,25.0,2.0,10.0
2,446183.40,4654067.06,276.64,88.0,2.0,10.0
3,446169.57,4654067.28,280.31,18.0,2.0,11.0
4,446158.14,4654067.66,279.39,51.0,2.0,11.0


In [12]:
#Transform to geopandas GeoDataFrame
crs = None
geometry = [Point(xy) for xy in zip(inFile.x,inFile.y)]

lidar_geodf = gpd.GeoDataFrame(lidar_df, crs=crs, geometry=geometry)
lidar_geodf.crs = {'init' :'epsg:4326'} # set correct spatial reference,elevation=elevations

In [14]:
elevations = [Point(xy) for xy in zip(inFile.x,inFile.y)]
elevations

 ...]

In [ ]:
lidar_geodf.head()

,x,y,z,intensity,raw_classification,scan_angle_rank,geometry
0,446179.84,4654067.44,277.06,96.0,2.0,10.0,POINT (446179.84000 4654067.44000)
1,446181.77,4654067.16,278.33,25.0,2.0,10.0,POINT (446181.77000 4654067.16000)
2,446183.40,4654067.06,276.64,88.0,2.0,10.0,POINT (446183.40000 4654067.06000)
3,446169.57,4654067.28,280.31,18.0,2.0,11.0,POINT (446169.57000 4654067.28000)
4,446158.14,4654067.66,279.39,51.0,2.0,11.0,POINT (446158.14000 4654067.66000)


In [ ]:
#turn feet to meters 
lidar_geodf = lidar_geodf.assign(x = lambda x:(x['x']*0.305))
lidar_geodf = lidar_geodf.assign(y = lambda x:(x['y']*0.305))
lidar_geodf = lidar_geodf.assign(z = lambda x:(x['z']*0.305))

In [ ]:
#select x,y,z and save to csv file
coordinates = lidar_geodf[['x','y','z']]
coordinates 
#save the first 2000 rows because of lack of computing power when rendering 3D
coordinates[:2000].to_csv('../Data/coodata_in_meters.csv')